<a href="https://www.kaggle.com/code/sudorenkoroma/gpt-text-clas?scriptVersionId=191123269" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer, TFGPT2ForSequenceClassification, create_optimizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from sklearn.metrics import classification_report
import os
import shutil

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1722759998.355148      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0804 08:26:38.363231096      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0804 08:26:38.363245699      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0804 08:26:38.363249072      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0804 08:26:38.363251514      13 

In [2]:
# Ініціалізація TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1722760011.391266      13 service.cc:145] XLA service 0x55cfb48f9c00 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722760011.391313      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1722760011.391317      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1722760011.391320      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1722760011.391323      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1722760011.391325      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1722760011.391328      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1722760011.391331      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1722760011.391333      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
# Load the dataset
file_path = '/kaggle/input/toxic-dataset/combined_train_big.csv'
data = pd.read_csv(file_path)

# Preprocess the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment_text'].tolist(),
    data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist(),
    test_size=0.1
)

# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add pad token and set padding side to left
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = "left"

# Tokenize function
MAX_LENGTH = 512

def tokenize_texts(texts, labels, tokenizer, max_length=MAX_LENGTH):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    return (
        np.array(input_ids),
        np.array(attention_masks),
        np.array(labels)
    )

# Tokenize the datasets
train_input_ids, train_attention_masks, train_labels = tokenize_texts(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_masks, val_labels = tokenize_texts(val_texts, val_labels, tokenizer)

# Create TensorFlow datasets
def create_tf_dataset(input_ids, attention_masks, labels, batch_size=16):
    dataset = Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks}, labels))
    dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

BATCH_SIZE = 32

train_dataset = create_tf_dataset(train_input_ids, train_attention_masks, train_labels, BATCH_SIZE)
val_dataset = create_tf_dataset(val_input_ids, val_attention_masks, val_labels, BATCH_SIZE)

In [4]:
from sklearn.utils.class_weight import compute_class_weight
#  Мітки
y_train = data.iloc[:, 2:]

# Ініціалізація словника для ваг класів
class_weights_dict = {}

# Обчислення ваг для кожного класу
for i, column in enumerate(y_train.columns):
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[column]), y=y_train[column])
    class_weights_dict[i] = class_weights[1]  # class_weight повертає ваги для обох класів 0 і 1, нас цікавить вага для класу 1

In [ ]:
EPOCHS = 20
with strategy.scope():
    # Load pre-trained model
    base_model = TFGPT2ForSequenceClassification.from_pretrained('gpt2')
    base_model.resize_token_embeddings(len(tokenizer))

    # Model definition using the last token for prediction
    input_ids = tf.keras.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='attention_mask')

    outputs = base_model.transformer(input_ids, attention_mask=attention_mask)
    last_token_output = outputs.last_hidden_state[:, -1, :]  # Get the last token's hidden state
    last_token_output_tanh = tf.keras.activations.tanh(last_token_output) 
    dropout1_output = tf.keras.layers.Dropout(0.3)(last_token_output_tanh)
    Dense_output = tf.keras.layers.Dense(64, activation='relu')(dropout1_output)
    final_output = tf.keras.layers.Dense(6, activation='sigmoid')(Dense_output)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=final_output)

    # Optimizer
    steps_per_epoch = len(train_texts) // BATCH_SIZE
    num_train_steps = steps_per_epoch * EPOCHS
    num_warmup_steps = int(0.1 * num_train_steps)
    optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=num_warmup_steps, num_train_steps=num_train_steps)
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Compilation of the model
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,           # Stop training if no improvement after 3 epochs
    restore_best_weights=True  # Restore the best model weights found during training
)

def train_model():
    with strategy.scope():
        # Training
        history = model.fit(
            train_dataset,
            epochs=EPOCHS,
            validation_data=val_dataset,
#             class_weight=class_weights_dict,
            callbacks=[early_stopping]  # Include early stopping callback here
        )

        # Evaluation of the model
        val_loss, val_acc = model.evaluate(val_dataset)
        print(f'Validation loss: {val_loss} accuracy: {val_acc}')

train_model()

I0000 00:00:1722760176.766727      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFGPT2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


2024-08-04 08:30:55.001226: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
I0000 00:00:1722760257.927019     814 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(a970e06d9437619c:0:0), session_name()
I0000 00:00:1722760287.201418     814 tpu_compile_op_common.cc:245] Compilation of a970e06d9437619c:0:0 with session name  took 29.274338671s and succeeded
I0000 00:00:1722760287.272448     814 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(a970e06d9437619c:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_train_function_9218482349578203974", property.function_library_fingerprint = 7450172369679775796, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, t

6287/6288 [============================>.] - ETA: 0s - loss: 0.1196 - accuracy: 0.7085

I0000 00:00:1722761068.506213     782 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(5e80787426413bd9:0:0), session_name()
I0000 00:00:1722761093.998878     782 tpu_compile_op_common.cc:245] Compilation of 5e80787426413bd9:0:0 with session name  took 25.492609156s and succeeded
I0000 00:00:1722761094.067406     782 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(5e80787426413bd9:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_train_function_9218482349578203974", property.function_library_fingerprint = 7450172369679775796, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,512,;2,512,;2,6,;", property.guaranteed_constants_size = 0, embedding_

6288/6288 [==============================] - ETA: 0s - loss: 0.1196 - accuracy: 0.7086

2024-08-04 08:45:03.220449: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
I0000 00:00:1722761103.863886     765 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(1d6785ab8bc6aeb0:0:0), session_name()
I0000 00:00:1722761107.213517     765 tpu_compile_op_common.cc:245] Compilation of 1d6785ab8bc6aeb0:0:0 with session name  took 3.349573089s and succeeded
I0000 00:00:1722761107.233025     765 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(1d6785ab8bc6aeb0:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_test_function_13331886682621282541", property.function_library_fingerprint = 5134677829110247893, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topolog

6288/6288 [==============================] - 928s 134ms/step - loss: 0.1196 - accuracy: 0.7086 - val_loss: 0.0615 - val_accuracy: 0.9951
Epoch 2/20
6288/6288 [==============================] - 804s 128ms/step - loss: 0.0514 - accuracy: 0.8853 - val_loss: 0.0471 - val_accuracy: 0.8389
Epoch 3/20
6288/6288 [==============================] - 803s 128ms/step - loss: 0.0443 - accuracy: 0.9008 - val_loss: 0.0448 - val_accuracy: 0.9941
Epoch 4/20
6288/6288 [==============================] - 803s 128ms/step - loss: 0.0396 - accuracy: 0.8908 - val_loss: 0.0427 - val_accuracy: 0.9881
Epoch 5/20
6288/6288 [==============================] - 802s 128ms/step - loss: 0.0358 - accuracy: 0.9072 - val_loss: 0.0445 - val_accuracy: 0.9888
Epoch 6/20
6288/6288 [==============================] - 803s 128ms/step - loss: 0.0323 - accuracy: 0.8212 - val_loss: 0.0459 - val_accuracy: 0.9777
Epoch 7/20
3481/6288 [===============>..............] - ETA: 5:47 - loss: 0.0283 - accuracy: 0.8033

In [7]:
predictions = []
true_labels = []

for batch in val_dataset:
    input_ids = batch[0]['input_ids']
    attention_mask = batch[0]['attention_mask']
    labels = batch[1]

    preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask}, verbose=0)

    predictions.extend(preds)
    true_labels.extend(labels.numpy())

predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Binarize predictions
predictions = np.where(predictions > 0.4, 1, 0)

# Print classification report
report = classification_report(true_labels, predictions, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=0)
print(report)


               precision    recall  f1-score   support

        toxic       0.74      0.83      0.78      2192
 severe_toxic       0.51      0.37      0.43       205
      obscene       0.79      0.84      0.81      1215
       threat       0.52      0.50      0.51        48
       insult       0.67      0.80      0.73      1106
identity_hate       0.50      0.68      0.58       198

    micro avg       0.72      0.80      0.76      4964
    macro avg       0.62      0.67      0.64      4964
 weighted avg       0.72      0.80      0.75      4964
  samples avg       0.07      0.08      0.07      4964

